In [1]:
import torch
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import importlib
import time
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import scipy.io
import matplotlib.pyplot as plt
from functools import partial
import time
import sys
import os
import gc
import importlib
import RFrtModel_SingleFracture_SolverDA_General as RF_1Frac_DA
import RFrtModel_SingleFracture_SolverDF as RF_1Frac_DF

In [3]:
importlib.reload(RF_1Frac_DA)
importlib.reload(RF_1Frac_DF)

<module 'RFrtModel_SingleFracture_SolverDF' from '/Users/huynh/Desktop/Python/UnitedFilter_SingleRFrt/General/RFrtModel_SingleFracture_SolverDF.py'>

In [5]:
def cond_alpha(t):
    # conditional information
    # alpha_t(0) = 1
    # alpha_t(1) = esp_alpha \approx 0
    return 1 - (1-eps_alpha)*t


def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def f(t):
    # f=d_(log_alpha)/dt
    alpha_t = cond_alpha(t)
    f_t = -(1-eps_alpha) / alpha_t
    return f_t

def g_sq(t):
    # g = d(sigma_t^2)/dt -2f sigma_t^2
    d_sigma_sq_dt = 1
    g2 = d_sigma_sq_dt - 2*f(t)*cond_sigma_sq(t)
    return g2

def g(t):
    return np.sqrt(g_sq(t))

def reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, n_dim, select_set, idA_sub, x0, time_steps, C,\
                score_likelihood=None, drift_fun=f, diffuse_fun=g, alpha_fun=cond_alpha, \
                sigma2_fun=cond_sigma_sq, save_path=False):
    # x_T: sample from standard Gaussian
    # x_0: target distribution to sample from

    # reverse SDE sampling process
    # N1 = x_T.shape[0]
    # N2 = x0.shape[0]
    # d = x_T.shape[1]

    # Generate the time mesh
    dt = 1.0/time_steps

    # Initialization
    xt = torch.randn(ensemble_size, n_dim, dtype=torch.float64, device=device)
    t = 1.0

    # define storage
    if save_path:
        path_all = [xt]
        t_vec = [t]

    # forward Euler sampling
    for i in range(time_steps):
        # prior score evaluation
        alpha_t = alpha_fun(t)
        sigma2_t = sigma2_fun(t)


        # Evaluate the diffusion term
        diffuse = diffuse_fun(t)

        # Evaluate the drift term
        # drift = drift_fun(t)*xt - diffuse**2 * score_eval

        # Update
        if score_likelihood is not None:
#             zt = score_likelihood(xt, t)
#             print(zt.size())
            xt += -dt*( drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t)-\
                       diffuse**2*score_likelihood(idA_sub, xt, t, C)) +np.sqrt(dt)*diffuse*torch.randn_like(xt)
    
        else:
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t))+np.sqrt(dt)*diffuse*torch.randn_like(xt)
        
        xt = xt.to(torch.float64)
        # Store the state in the path
        if save_path:
            path_all.append(xt)
            t_vec.append(t)

        # update time
        t = t - dt

    if save_path:
        return path_all, t_vec
    else:
        return xt

In [7]:
def systematic_resample_pytorch(weights):
    N = len(weights)
    
    # Generate positions with a consistent random offset
    positions = (torch.rand(1) + torch.arange(N)) / N

    # Compute cumulative sum of weights
    cumulative_sum = torch.cumsum(weights, dim=0)

    # Initialize the indexes tensor
    indexes = torch.zeros(N, dtype=torch.long)
    
    # Initialize pointers
    i, j = 0, 0
    while i < N and j < len(cumulative_sum):
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    
    return indexes

In [9]:
xt = 0
xe = 2
yt = 0
ye = 1
nx = 80
ny = 40
node_frt = ny + 1
device = 'cpu'
# device = 'cuda'
nt = 800
T = 2
t0 = 0
SDE_sigma = 0.005

h = (ye - yt) / ny
dt = (T - t0) / nt

frt_length = 1e-03

timegrid = torch.linspace(0, T, nt + 1)
poros_subdom = 1
poros_gamma = 1 / frt_length
poros_frt = frt_length * poros_gamma

# Mesh generation and diffusion parameters
TriMesh_Subdom = RF_1Frac_DA.MeshGenerator_PureDiff(xt, xe, yt, ye, nx, ny, device)
n_dim = TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1
DOFs_Local = [None, None]
NumEgs = [None, None]
NumEms = [None, None]
for k in range(2):
    DOFs_Local[k] = TriMesh_Subdom[k].DOFs
    NumEgs[k] = TriMesh_Subdom[k].NumEgs
    NumEms[k] = TriMesh_Subdom[k].NumEms
# print(TriMesh_Subdom[0].DirichletEgs.shape)

DOFs = DOFs_Local[0]+DOFs_Local[1]+node_frt+ny
diff_1subdom = 1
diff_2subdom = 1

diff_frt = torch.zeros(ny, dtype=torch.float64)

# Assign values to segments of diff_frt
diff_frt[:ny // 4] = 1.0
diff_frt[ny // 4:3 * ny // 4] = 0.002
diff_frt[3 * ny // 4:] = 1.0

# Calculate keppa and alp_frt
keppa = (2 * diff_frt) / frt_length
alp_frt = diff_frt * frt_length

diffusion_ref = RF_1Frac_DA.compute_diffusion_single(diff_1subdom, diff_2subdom, TriMesh_Subdom, device)

GAUSSQUAD = RF_1Frac_DA.SetGaussQuad(2, 4, 3, device)
Eqn_Subdom = [RF_1Frac_DA.EqnBC(), RF_1Frac_DA.EqnBC()]
EqnBC_fracture = RF_1Frac_DA.EqnBC_frt()
Local_Mat = [None]*2
Local_Mat_pertb = [None]*2

for k in range(2):
    Local_Mat[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat(TriMesh_Subdom[k], dt, diffusion_ref[k], poros_subdom, device)
    
MatB = [None]*2
MatC = [None]*2

MatATest = [None]*2
Local_MatTest = [None]*2

for k in range(2):
    MatB[k], MatC[k] = RF_1Frac_DA.ComputeBandCblocks(TriMesh_Subdom[k], dt, poros_subdom,  device)
    
for k in range(2):
    MatATest[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dt, diffusion_ref[k], device)
    Local_MatTest[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatATest[k], MatB[k], MatC[k], dt)    

Glob_MatTest = RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                                Local_MatTest, alp_frt, keppa, dt, \
                                                                poros_frt, device)

BCs_frt = 3
BCs_Subdom = 3
X0 = torch.zeros((1, TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1), dtype=torch.float64, device=device)

X0 = X0.to(device)
state_target = X0
state_ref = torch.zeros(nt+1, n_dim, device = device)
state_ref[[0], :] += state_target

for i in range(nt):
    print(i)
    noise2 = np.sqrt(dt)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, 1),\
                                                                dtype=torch.float64, device=device))
    stateT = torch.transpose(state_ref[[i], :], 0, 1)
    sln = RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General(stateT, ny, node_frt, TriMesh_Subdom, nt, \
                                                                   dt, poros_subdom, poros_frt, BCs_frt, BCs_Subdom, \
                                                                   Glob_MatTest, GAUSSQUAD, Eqn_Subdom, \
                                                                   EqnBC_fracture, SDE_sigma, device, noise2, 0)
    state_ref[[i+1], :] += torch.transpose(sln, 0, 1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
Mat_save = Glob_MatTest.clone()
Mat_save = Mat_save.to_dense()
Mat_save = Mat_save.numpy()

RHS_save = GlbRHS_cpu.numpy()
scipy.io.savemat('TestMat_PureDiff_General.mat', {'TestMat':Mat_save, 'TestRHS':RHS_save})

In [11]:
state_ref_save = state_ref.clone()
state_ref_save = state_ref.cpu().numpy()

scipy.io.savemat('TestSol_PureDiff_General_Ref_v1.mat', {'ExactState':state_ref_save})

In [120]:
TotalEms = TriMesh_Subdom[0].NumEms+TriMesh_Subdom[1].NumEms+ny
TotalEgs = TriMesh_Subdom[0].NumEgs+TriMesh_Subdom[1].NumEgs+node_frt
#### 50%
indices_A = torch.randperm(TotalEgs)[:int(0.5 * TotalEgs)]

# Generate indices for 75% of the second range (A+1 to A+B)
indices_B = torch.randperm(TotalEms)[:int(0.5 * TotalEms)] + TotalEgs

spa_indices, _ = torch.sort(torch.cat([indices_A, indices_B]))

num_indices= indices_A.shape[0]+indices_B.shape[0]

r = indices_B.shape[0] / indices_A.shape[0]
x = 50*(r+1)

idA_sub = torch.randperm(len(indices_A))[:int(round((x / 100) * len(indices_A)))]


# # Sort the selected indices in increasing order
idA_sub, _ = torch.sort(idA_sub)

In [15]:
idA_sub_save = idA_sub.clone()
idA_sub_save = idA_sub_save.numpy()

spa_indices_save = spa_indices.clone()
spa_indices_save = spa_indices_save.numpy()
scipy.io.savemat('SpatialSelection_Generalv1.mat', {'ida_sub_save':idA_sub_save, 'spa_indices_save':spa_indices_save})

In [132]:
# data = scipy.io.loadmat('TestUnitedSF_RefSol_v1_NewBCs_T5.mat')
# state_ref = torch.from_numpy(data['ExactState'])

# data = scipy.io.loadmat('SpatialSelection_Generalv1.mat')
# idA_sub = torch.from_numpy(data['ida_sub_save'])
# spa_indices = torch.from_numpy(data['spa_indices_save'])

# idA_sub = idA_sub.squeeze(0)
# spa_indices = spa_indices.squeeze(0)
# ida_sub = ida_sub.to(torch.int64)
# spa_indices = spa_indices.to(torch.int64)
# filtering_steps = nt
n_dim = TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1

ntEnSF = 50
filtering_steps = ntEnSF
timeTrue = torch.linspace(0, 1, nt+1)
tEnSF = torch.linspace(0, 1, filtering_steps+1)
indices = torch.searchsorted(timeTrue, tEnSF, right=False)

state_EnSF = state_ref[indices, :].clone()

state_EnSF_Swap = RF_1Frac_DA.RearrangeSol(state_EnSF, TriMesh_Subdom, ny, node_frt)
dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.2
####################################################################
# EnSF setup
# define the diffusion process
eps_alpha = 0.05

# ensemble size
ensemble_size = 200
ensemble_true = 1
# forward Euler step
euler_steps = 200
sub_steps = 4

time_euler = torch.linspace(0, 1, euler_steps+1)
time_sub = torch.linspace(0, 1, sub_steps+1)
select_set = torch.searchsorted(time_euler, time_sub, right=False)

print(select_set)
# device = 'cpu'
# device = 'cuda'

def g_tau(t):
    return 1-t

# saving file name
exp_name = 'JointStatePara_RFrtModel_MixedRedo_General'

x_state = 5*torch.randn(ensemble_size, n_dim, device=device)
# x_state = x_state.to(torch.float16)
mem_state = state_target.element_size() * state_target.nelement()/1e+6
mem_ensemble = mem_state * ensemble_size
print(f'single state memory: {mem_state:.2f} MB')
print(f'state ensemble memory: {mem_ensemble:.2f} MB')

rmse_all = []
obs_save = []
est_save = torch.zeros(filtering_steps+1, n_dim, device = device)
est_save[[0], :] += torch.mean(x_state,dim=0)

x0filter = x_state
theta0 = torch.tensor([8, 8, 100, 8], dtype=torch.float64, device =device); #Initial guess
n = 36;                     # Number of particles considered     
Cov = torch.eye(4, dtype=torch.float64, device =device) * torch.tensor([4, 3, 50, 2], dtype=torch.float64, device =device)
l= 3
particles =torch.distributions.MultivariateNormal(theta0, Cov).sample((n,))
xs = [theta0]
weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
IntmediaPara = torch.sum(particles * weights.unsqueeze(1), dim=0) 
# SlnRefT = torch.transpose(ExtractSlnRef, 0, 1)
X = torch.zeros((ntEnSF+1, DOFs), dtype=torch.float64, device=device) #545 is the dof in the mesh considered
X[0, :] += state_EnSF[0, :]
for i in range(1, ntEnSF+1):
    X[i, :] += state_EnSF[i, :]
    
state_United = state_EnSF_Swap[:, spa_indices].clone()

state_United = state_United.to(torch.float64)
factorR = 0.36
R = factorR*torch.eye(DOFs, dtype=torch.float64, device=device)   # Covariance considered 
cholR = torch.linalg.cholesky(R)
factor = 100
R_scaled = factor * R
Z = cholR[torch.meshgrid(spa_indices, spa_indices, indexing ='ij')] @ \
    torch.randn(num_indices, ntEnSF+1, dtype=torch.float64, device=device)

state_United += torch.transpose(Z, 0, 1)

NeumannEgs = [None]*2
for k in range(2):
    NeumannEgs[k] = TriMesh_Subdom[k].NeumannEgs

LocalMat_DF = [None]*2

MatD = RF_1Frac_DA.ComputeDblocks(TriMesh_Subdom, h, dtEnSF, ny, device)

tensor([  0,  50, 100, 150, 200])
single state memory: 0.13 MB
state ensemble memory: 25.99 MB


In [134]:
print(IntmediaPara)

tensor([  7.7829,   7.9684, 100.8409,   8.3549], dtype=torch.float64)


In [136]:
for i in range(filtering_steps):
# for i in range(2):
    # weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
    print(f'step={i}:')
    t1 = time.time()
    obs = state_United[[i+1], :].clone()
    obs[:, idA_sub] = torch.atan(state_United[[i+1], idA_sub].clone())
    obs += torch.randn_like(state_United[[i+1], :])*obs_sigma
#     obs = state_United[[i+1], :].clone()+torch.randn_like(state_United[[i+1], :])*obs_sigma
#     obs = torch.atan(state_United[[i+1], :].clone())+torch.randn_like(state_United[[i+1], :])*obs_sigma
    def score_likelihood(idA_sub, xt, t, C):
        # obs: (d)
        # xt: (ensemble, d)
#         score_x = -(torch.atan(xt)/5 - obs)/obs_sigma**2 * (1./(5*(1+xt**2)))
        A = -(xt-obs)/obs_sigma**2
        score_x = A.clone()
        score_x[:, idA_sub] = -(torch.atan(xt[:, idA_sub]) - obs[:, idA_sub])/obs_sigma**2 * (1./(1+xt[:, idA_sub]**2))
#         score_x = -(torch.atan(xt) - obs)/obs_sigma**2 * (1./(1+xt**2))
        tau = g_tau(t)
        return tau*score_x/C
    
    x0EnSF = x_state.clone()
    x0filterT = torch.transpose(x0EnSF.clone(), 0, 1)
    for j in range(l):
        weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
        IntmediaPara_Start = IntmediaPara.clone()
        ##### EnSF 
        Local_Matloop = [None]*2
        MatAEnSF = [None]*2
        
        diffusion_loopEnSF = RF_1Frac_DA.compute_diffusion_single(IntmediaPara_Start[0] / 1, \
                                                                  IntmediaPara_Start[3] / 1, TriMesh_Subdom, device)
        for k in range(2):
            MatAEnSF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diffusion_loopEnSF[k], device)
            Local_Matloop[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatAEnSF[k], MatB[k], MatC[k], dtEnSF)

        diff_frtDA = torch.zeros(ny, dtype=torch.float64)

        # Assign values to segments of diff_frt
        diff_frtDA[:ny // 4] = IntmediaPara_Start[1]
        diff_frtDA[ny // 4:3 * ny // 4] = 1 / IntmediaPara_Start[2]
        diff_frtDA[3 * ny // 4:] = IntmediaPara_Start[1]
        
        # Calculate keppa and alp_frt
        keppaDA = (2 * diff_frtDA) / frt_length
        alp_frtDA = diff_frtDA * frt_length
        
        Glob_Mat_loop = RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                                         Local_Matloop, alp_frtDA, keppaDA, dtEnSF, \
                                                                         poros_frt, device)
        
        noise1 = np.sqrt(dtEnSF)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, ensemble_size),\
                                                                        dtype=torch.float64, device=device))

        # x0filterT = torch.transpose(x0EnSF.clone(), 0, 1)
        sln_state = RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General(x0filterT, ny, node_frt, TriMesh_Subdom, ntEnSF, \
                                                                             dtEnSF, poros_subdom, poros_frt, BCs_frt, BCs_Subdom, \
                                                                             Glob_Mat_loop, GAUSSQUAD, Eqn_Subdom, EqnBC_fracture,\
                                                                             SDE_sigma, device, noise1, 0)
        
        x_state = torch.transpose(sln_state, 0, 1)
        x_state = RF_1Frac_DA.RearrangeSol(x_state, TriMesh_Subdom, ny, node_frt)
        
        sln_bar = reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, num_indices, select_set, idA_sub,\
                              x0=x_state[:, spa_indices], time_steps=euler_steps, C=1, \
                              score_likelihood=score_likelihood)
        
        x_state[:, spa_indices] = sln_bar.clone()
    
        x_state[:, TotalEgs + torch.arange(0, NumEms[0])] =\
            torch.clamp(x_state[:, TotalEgs + torch.arange(0, NumEms[0])],\
                        min=0, max = BCs_Subdom)

        x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])] =\
                    torch.clamp(x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])],\
                                min=0, max = BCs_Subdom)

        x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)] =\
                    torch.clamp(x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)],\
                                min=0, max = 1.5)
        x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-4, max=3.4)
        x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-3, max=4)
        x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
                        min=-3e-05, max = 3e-05)
        # if i < int(filtering_steps / 2):
        #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-20, max=20)
        #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-25, max=20)
        #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
        #                     min=3, max = 18)
        # elif int(filtering_steps / 2) < i and i <int(3*filtering_steps / 5):
        #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-15, max=10)
        #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-20, max=10)
        #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
        #                     min=3, max = 11)
        # elif int(3*filtering_steps / 5) < i and i <int(4*filtering_steps / 5):
        #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-13, max=10)
        #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-15, max=10)
        #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
        #                     min=3, max = 11)
        # elif i > int(4*filtering_steps / 5):
        #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-10, max=10)
        #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-10, max=10)
        #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = \
        #                     torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)], min=5, max = 8)
            
        x_state[:, NeumannEgs[0]] = 0
        x_state[:, NumEgs[0]+NeumannEgs[1]] = 0
        x_state[:, NumEgs[0]+NumEgs[1]] = 0
        x_state[:, NumEgs[0]+NumEgs[1]+ny] =0
        
        x_state = RF_1Frac_DA.RearrangeSol_Reverse(x_state, TriMesh_Subdom, ny, node_frt)
        ####     
        xstate_bar = torch.mean(x_state,dim=0) # \bar{X}^{(l)}_{i+1}

        # factor = torch.tensor([1, 1, 1, 1], dtype=torch.float64)
        threshold1 = torch.tensor([3, 5, 350, 5], dtype=torch.float64)
        threshold2 = torch.tensor([1.3, 1.6, 460, 1.3], dtype=torch.float64)
        
        Ep = torch.diag(torch.tensor([0.9, 0.49, 500, 0.49], dtype=torch.float64))      
        replacement_values1 = torch.tensor([0.64, 0.16, 360, 0.16], dtype=torch.float64)  # Example replacement values
        replacement_values2 = torch.tensor([0.009, 0.006, 20,  0.005], dtype=torch.float64)  # Example replacement values
        for jj in range(len(IntmediaPara_Start)):
            ajj = IntmediaPara_Start[jj] 
            if jj != 2:
                if ajj < threshold1[jj] and ajj > threshold2[jj]:  # Condition to check
                    print(jj)
                    print(ajj)
                    print(f"Replacing diagonal entry {jj} of Ep by 1st")
                    Ep[jj, jj] = replacement_values1[jj]  # Replace the diagonal entry 
                elif ajj < threshold2[jj]:  # Condition to check
                    print(jj)
                    print(ajj)
                    print(f"Replacing diagonal entry {jj} of Ep by 2nd")
                    Ep[jj, jj] = replacement_values2[jj]  # Replace the diagonal entry 
            elif jj == 2:
                if ajj > threshold1[jj] and ajj < threshold2[jj]:  # Condition to check
                    print(f"Replacing diagonal entry {jj} of Ep by 1st")
                    Ep[jj, jj] = replacement_values1[jj]  # Replace the diagonal entry 
                elif ajj > threshold2[jj]:  # Condition to check
                    print(f"Replacing diagonal entry {jj} of Ep by 2nd")
                    Ep[jj, jj] = replacement_values2[jj]  # Replace the diagonal entry
        
        
        # Check if i is in the desired range
    # Replace diagonal values based on IntmediaPara_Start
    
   
        cholEp = torch.linalg.cholesky(Ep)
        particles += torch.randn(n, 4, dtype=torch.float64)@cholEp #\tilde{\gamma}^{k, (l+1)}_{i+1}
        # print(particles[:, 0])
        print(particles[:, 1])
        print(particles[:, 2])
        # print(particles[:, 3])
        for qq in range(particles.shape[1]):  # Iterate over columns
            condition = particles[:, qq] < 0  # Check for negative values in the column
            if condition.any():  # If there are negative values
                print(qq)
                print(f"negative detected in {'d2' if qq == 1 else 'd1' if qq == 0 else 'd3'}")
                # Find the smallest positive entry in the column
                positive_values = particles[:, qq][particles[:, qq] > 0]
                if positive_values.numel() > 0:  # Ensure there are positive values
                    smallest_positive = positive_values.min()
                    # Replace negative values with the smallest positive entry
                    particles[:, qq] = torch.where(
                        condition,
                        smallest_positive,
                        particles[:, qq]
                    )
#         z = xstate_bar.clone() #    

        z = xstate_bar.clone() # 
        P = torch.zeros(n, DOFs, dtype=torch.float64, device=device)

        x0DFT = torch.transpose(est_save[[i], :].clone(), 0, 1)
     
        for o, particle in enumerate(particles):            
            MatADF = [None]*2
            diff_DF = RF_1Frac_DA.compute_diffusion_single(particle[0], particle[3], TriMesh_Subdom, device)
  
            for k in range(2):
                MatADF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diff_DF[k], device)
                LocalMat_DF[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatADF[k], MatB[k], MatC[k], dtEnSF) 

            diff_frtDF = torch.zeros(ny, dtype=torch.float64)

            # Assign values to segments of diff_frt
            diff_frtDF[:ny // 4] = particle[1]
            diff_frtDF[ny // 4:3 * ny // 4] = 1 / particle[2]
            diff_frtDF[3 * ny // 4:] = particle[1]
            
            # Calculate keppa and alp_frt
            keppaDF = (2 * diff_frtDF) / frt_length
            alp_frtDF = diff_frtDF * frt_length
        
            GlobMatDF = \
            RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                             LocalMat_DF, alp_frtDF, keppaDF, dtEnSF, \
                                                             poros_frt, device)
            
            SlnDF = \
            RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General(x0DFT, ny, node_frt, TriMesh_Subdom, ntEnSF, \
                                                                     dtEnSF, poros_subdom, poros_frt, BCs_frt, BCs_Subdom, \
                                                                     GlobMatDF, GAUSSQUAD, Eqn_Subdom, EqnBC_fracture,\
                                                                     SDE_sigma, device, noise1, 0)
            SlnDFT = torch.transpose(SlnDF, 0, 1)
            P[o, :] += SlnDFT[-1, :] # \bar{X}^{\tilde{\gamma}^{k, (l+1)}_{i+1}}_{i+1}

        P -=  z.unsqueeze(0)   # Broadcasting is already efficient

        # Step 2: Solve R * X = P^T for X
        Rsolved = P*(1/factorR)
        Rnorm = torch.sum(Rsolved * P, dim=1)
        Rnorm_clone = Rnorm.clone() / 1
        
        print(Rnorm_clone)
        
        if torch.min(Rnorm_clone) > 1200:
            alp_factor = torch.min(Rnorm_clone) / 1200 
#             print(alp_factor)
            Rnorm_clone /= alp_factor 
                    
        # print(Rnorm_clone)

        # Step 4: Update weights
        weights.mul_(torch.exp(-0.5 * Rnorm_clone))  # In-place operation
#         print(weights)
        weights.add_(1e-300)  # Add directly without creating new tensors
        weights.div_(weights.sum())  # Normalize in-place
#         print(weights)
        # Step 5: Resampling and updating particles
        indexes = systematic_resample_pytorch(weights)
        particles[:] = particles[indexes]
        # print(particles[:, 0])
        print(particles[:, 1])
        print(particles[:, 2])
        # print(particles[:, 3])
        # Step 6: Update mean
        mu = torch.sum(particles * weights.unsqueeze(1), dim=0)
        IntmediaPara = mu.clone() 
        # print(IntmediaPara)
        print(IntmediaPara / torch.tensor([1, 1, 1, 1]))
    
    ## EnSF with \bar{\gamma}^{L}_{i+1}
    IntmediaPara_lastEnSF = IntmediaPara.clone()
    diffusion_ref_pertb = RF_1Frac_DA.compute_diffusion_single(IntmediaPara_lastEnSF[0],\
                                                               IntmediaPara_lastEnSF[3], TriMesh_Subdom, device)
    for k in range(2):
        MatAEnSF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diffusion_ref_pertb[k], device)
        Local_Mat_pertb[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatAEnSF[k], MatB[k], MatC[k], dtEnSF)

    diff_frtlastDA = torch.zeros(ny, dtype=torch.float64)

    # Assign values to segments of diff_frt
    diff_frtlastDA[:ny // 4] = IntmediaPara_lastEnSF[1]
    diff_frtlastDA[ny // 4:3 * ny // 4] = 1 / IntmediaPara_lastEnSF[2]
    diff_frtlastDA[3 * ny // 4:] = IntmediaPara_lastEnSF[1]
    
    # Calculate keppa and alp_frt
    keppalastDA = (2 * diff_frtlastDA) / frt_length
    alp_frtlastDA = diff_frtlastDA * frt_length

    Glob_Mat_pertb = \
    RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                     Local_Mat_pertb, alp_frtlastDA, keppalastDA, dtEnSF, \
                                                     poros_frt, device)

    noise1 = np.sqrt(dtEnSF)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, ensemble_size),\
                                                                    dtype=torch.float64, device=device))

    x0filterT_last = torch.transpose(x0EnSF.clone(), 0, 1)
    sln_state = RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General(x0filterT_last, ny, node_frt, TriMesh_Subdom, ntEnSF, \
                                                                         dtEnSF, poros_subdom, poros_frt, BCs_frt, BCs_Subdom,\
                                                                         Glob_Mat_pertb, GAUSSQUAD, Eqn_Subdom, EqnBC_fracture,\
                                                                         SDE_sigma, device, noise1, 0)
    
    x_state = torch.transpose(sln_state, 0, 1)
    x_state = RF_1Frac_DA.RearrangeSol(x_state, TriMesh_Subdom, ny, node_frt)
    
    sln_bar = reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, num_indices, select_set, idA_sub,\
                          x0=x_state[:, spa_indices], time_steps=euler_steps, C=1, \
                          score_likelihood=score_likelihood)
    x_state[:, spa_indices] = sln_bar.clone()

    x_state[:, TotalEgs + torch.arange(0, NumEms[0])] =\
            torch.clamp(x_state[:, TotalEgs + torch.arange(0, NumEms[0])],\
                        min=0, max = BCs_Subdom)

    x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])] =\
                torch.clamp(x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])],\
                            min=0, max = BCs_Subdom)

    x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)] =\
                torch.clamp(x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)],\
                            min=0, max = 1.5)
    
    x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-4, max=3.4)
    x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-4, max=4)
    x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
                    min=-3e-05, max = 3e-05)
    # if i < int(filtering_steps / 2):
    #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-20, max=20)
    #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-25, max=20)
    #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
    #                     min=3, max = 18)
    # elif int(filtering_steps / 2) < i and i <int(3*filtering_steps / 5):
    #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-15, max=10)
    #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-20, max=10)
    #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
    #                     min=3, max = 11)
    # elif int(3*filtering_steps / 5) < i and i <int(4*filtering_steps / 5):
    #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-13, max=10)
    #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-15, max=10)
    #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
    #                     min=3, max = 11)
    # elif i > int(4*filtering_steps / 5):
    #     x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-10, max=10)
    #     x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-10, max=10)
    #     x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = \
                        # torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)], min=5, max = 8)
        
    x_state[:, NeumannEgs[0]] = 0
    x_state[:, NumEgs[0]+NeumannEgs[1]] = 0
    x_state[:, NumEgs[0]+NumEgs[1]] = 0
    x_state[:, NumEgs[0]+NumEgs[1]+ny] =0
    x_state = RF_1Frac_DA.RearrangeSol_Reverse(x_state, TriMesh_Subdom, ny, node_frt)
    
    x_statebarEnSF = torch.zeros_like(est_save[[i+1], :])
    x_statebarEnSF += torch.mean(x_state.clone(),dim=0) #\bar{X^{L}}_{i+1} = \bar{X}_{i+1}
    
    
    est_save[[i+1], :] += torch.mean(x_state.clone(),dim=0)
    xs.append(mu)
    diff = torch.sqrt(torch.mean((est_save[[i+1], :]  - x_statebarEnSF)**2)).item()
    rmse_temp = torch.sqrt(torch.mean((est_save[[i+1], :]  - state_EnSF[[i+1], :])**2)).item()
#     rmse_temp = torch.sqrt(torch.mean((est_save[[i+1], :]  - state_EnSF[[i+1], :])**2)).item()
    rmse = torch.sqrt(torch.mean((x_statebarEnSF  - state_EnSF[[i+1], :])**2)).item()
    
    # get time
    if x_state.device.type == 'cuda':
        torch.cuda.current_stream().synchronize()
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2-t1:.4f} ')

    # save information
    rmse_all.append(rmse_temp)
    # break
    if rmse_temp > 1000:
        print('diverge!')
        break

# est_save = RF_1Frac_DA.RearrangeSol_Reverse(est_save, TriMesh_Subdom, ny, node_frt)

state_savepy  = state_ref.clone()
est_savepy = est_save.clone()

state_savepy = state_savepy.cpu().numpy()
est_savepy = est_savepy.cpu().numpy()
rmse_all = np.array(rmse_all)

scipy.io.savemat('UnitedFilter_2DRft_50PartialOb_Mixed_General_v3.mat',\
                 {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
                 'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})

step=0:
tensor([ 8.7822,  9.0837,  6.4459,  6.8598,  7.1543,  8.8038, 12.0685, 10.0802,
         9.8565,  7.3572,  8.4790,  8.3864,  7.4143,  7.2277,  9.4316,  5.6379,
         3.7632,  6.3705,  9.4946,  9.5705,  9.6628,  7.8543,  8.5360,  7.7593,
         9.0580,  8.6753,  5.9340,  7.4425,  6.8799,  4.2774,  7.4770, 10.4564,
         8.5543,  5.4952,  8.4027, 11.3624], dtype=torch.float64)
tensor([140.5228, 106.0594, 126.8120, 121.0849,  40.2998,  93.3148,  86.5350,
         39.0377,  93.8319, 150.1108, 111.4078,  87.9879, 124.3144,  96.6031,
         83.0645, 108.9250,  68.2675,  91.9968, 120.5403, 108.3725, 128.9274,
        129.0245,  76.0849,  93.6055,  83.3283, 103.1708,  90.3292,  90.5824,
         76.8261,  64.6739, 111.6805,  79.3575,  78.4716, 139.2584, 111.7340,
        108.7211], dtype=torch.float64)
tensor([3161405.5999, 1174174.2231, 4719127.4585, 4980669.9673, 4589883.0113,
        2957449.2361, 3952618.3772, 1872041.9578, 4566640.7545, 3279167.3905,
        2499563.4333

KeyboardInterrupt: 